In [1]:
import sys
sys.path.append("C:/Users/jnevin/Documents/GitHub/networkdiffusionanalyser")

from netdiffanalyse.datahandling.dataintegration_updated import FeatureSetup, MatchClassifierFit, NetworkIntegration

In [2]:
import pandas as pd
import numpy as np
import networkx as nx

test_df = pd.read_csv('test_df.csv')
test_df.index = test_df.rec_id
test_df = test_df.drop(columns = ['rec_id'])

test_matches = np.load('test_matches.npy', allow_pickle = True)
test_matches = pd.MultiIndex.from_tuples(test_matches)

test_graph = nx.read_gml('test_graph.gml')

In [3]:
test_blocks = {'Block': [['given_name', 'given_name']]}
test_compares = {'Exact':  [['given_name', 'given_name'], ['date_of_birth', 'date_of_birth'],
                          ['suburb', 'suburb'], ['state', 'state']],
                'String': [['surname', 'surname', 'jarowinkler', 0.85], ['address_1', 'address_1', 'levenshtein', 0.85]]}

In [4]:
test_feature_setup = FeatureSetup(test_blocks, test_compares, test_df)

In [5]:
test_feature_setup.initialise_indexer()
test_feature_setup.get_candidate_links()

In [6]:
len(test_feature_setup.candidate_links) # 2082

2082

In [7]:
test_feature_setup.initialise_comparer()
test_feature_setup.calculate_features()

In [8]:
test_feature_setup.features.sum(axis=1).value_counts().sort_index(ascending=False) # 142, 145, 30, 9, 376, 1380

6.0     142
5.0     145
4.0      30
3.0       9
2.0     376
1.0    1380
dtype: int64

In [9]:
test_classifier = MatchClassifierFit('NaiveBayesClassifier', test_feature_setup.features, test_matches)
test_classifier.fit_model()

In [10]:
test_classifier.pred_matches(test_feature_setup.features)

MultiIndex([('rec-122-dup-0',   'rec-122-org'),
            (  'rec-183-org', 'rec-183-dup-0'),
            ('rec-248-dup-0',   'rec-248-org'),
            ('rec-469-dup-0',   'rec-469-org'),
            ('rec-373-dup-0',   'rec-373-org'),
            (   'rec-10-org',  'rec-10-dup-0'),
            ('rec-342-dup-0',   'rec-342-org'),
            (  'rec-397-org', 'rec-397-dup-0'),
            (  'rec-472-org', 'rec-472-dup-0'),
            (  'rec-330-org', 'rec-330-dup-0'),
            ...
            (  'rec-5-dup-0',     'rec-5-org'),
            ('rec-407-dup-0',   'rec-407-org'),
            ('rec-367-dup-0',   'rec-367-org'),
            ('rec-103-dup-0',   'rec-103-org'),
            ('rec-195-dup-0',   'rec-195-org'),
            ('rec-184-dup-0',   'rec-184-org'),
            (  'rec-252-org', 'rec-252-dup-0'),
            ( 'rec-48-dup-0',    'rec-48-org'),
            ('rec-298-dup-0',   'rec-298-org'),
            (  'rec-282-org', 'rec-282-dup-0')],
           names=['rec_

In [11]:
test_network_integrator = NetworkIntegration(test_graph, test_classifier.pred_matches(test_feature_setup.features),
                                             'walktrap_integration')

In [12]:
test_network_integrator.integrate_network()

In [14]:
org_graph = test_network_integrator.graph
adj_graph = test_network_integrator.adj_graph

In [16]:
len(adj_graph.nodes()) # 646

646

In [17]:
len(org_graph.nodes()) # 922

922